# More on Functional Programming

## Using higher-order functions


Collecting results of functions

In [1]:
; A function that returns the square of each element of a list

(defn square [x]
    (* x x))

(defn square-all [numbers]
    (if (empty? numbers)
        nil 
    (cons (square (first numbers))
          (square-all (rest numbers)))))

(square-all [1 2 3 4 5 6])


(1 4 9 16 25 36)

In [2]:
; A function that returns the cube of each element of a list

(defn cube [x]
    (* x x x))

(defn cube-all [numbers]
    (if (empty? numbers)
        ()
    (cons (cube (first numbers))
          (cube-all (rest numbers)))))

(cube-all [1 2 3 4 5 6])

(1 8 27 64 125 216)

In [3]:
; A HOF 'do-to-all' that generalizes tha las 2 functions

(defn do-to-all [func numbers]
    (if (empty? numbers)
        ()
    (cons (func (first numbers))
          (do-to-all func (rest numbers)))))


#'user/do-to-all

In [4]:
; The new function 'do-to-all' can get the square of each element

(do-to-all square [1 2 3 4 5 6])

(1 4 9 16 25 36)

In [5]:
; The new function 'do-to-all' can get the cube of each element

(do-to-all cube [1 2 3 4 5 6])


(1 8 27 64 125 216)

In [7]:
; In summary, 'do-to-all' is a custom implementation of 'map'

(map cube [1 2 3 4 5 6])


(1 8 27 64 125 216)

In [8]:
; But 'do-to-all' blows the stack with large numbers

(do-to-all square (range 11000))


Execution error (StackOverflowError) at user/do-to-all (REPL:7).
null


class java.lang.StackOverflowError: 

In [11]:
; A new implementation of 'do-to-all' using lazy sequences

(defn do-to-all-2 [func numbers]
    (lazy-seq 
        (if (empty? numbers)
            ()
        (cons (func (first numbers))
              (do-to-all func (rest numbers))))))


#'user/do-to-all-2

In [16]:
; Testing with a large number again (doesn't blow the stack anymore)

(take 10 (drop 10000 (do-to-all-2 square (range 11000))))


(100000000 100020001 100040004 100060009 100080016 100100025 100120036 100140049 100160064 100180081)

Reduce list of things

In [17]:
; A function that returns the sum of a list of numbers

(defn total-of [numbers]
    (loop [nums numbers sum 0]
        (if (empty? nums)
            sum
            (recur (rest nums) (+ sum (first nums))))))

(total-of [5 7 9 3 4 1 2 8])

39

In [20]:
; A couple of functions that returns the biggest number of a list of numbers

(defn larger-of [x y]
    (if (> x y) x y))

(defn largest-of [numbers]
    (loop [l numbers candidate (first numbers)]
        (if (empty? l)
            candidate
            (recur (rest l) (larger-of candidate (first l))))))

(largest-of [5 7 9 3 4 1 2 8])

9

In [22]:
; A HOF 'compute-across' that generalizes the last 2 functions

(defn compute-across [func elements value]
    (if (empty? elements)
        value
        (recur func (rest elements) (func value (first elements)))))


#'user/compute-across

In [23]:
; 'compute-across' can compute the sum of a list of numbers

(defn total-of-2 [numbers]
    (compute-across + numbers 0))

(total-of-2 [5 7 9 3 4 1 2 8])

39

In [25]:
; 'compute-across' can also get the biggest number of a list of numbers

(defn largest-of-2 [numbers]
    (compute-across larger-of numbers (first numbers)))

(largest-of [5 7 9 3 4 1 2 8])


9

In [39]:
; ; In summary, 'compute-across' is a custom implementation of 'reduce'

(reduce + [5 7 9 3 4 1 2 8])

39

In [40]:
(reduce max [5 7 9 3 4 1 2 8])


9

Filtering lists of things


In [44]:
; A function that returns a list of the numbers greater than a given number

(defn all-greater-than [threshold numbers]
    (compute-across #(if (> %2 threshold) (conj %1 %2) %1) numbers [])) ; '%' are the anon function args

(all-greater-than 5 [5 7 9 3 4 1 2 8])

[7 9 8]

In [46]:
; A function that returns a list of the numbers smaller than a given number

(defn all-lesser-than [threshold numbers]
    (compute-across #(if (< %2 threshold) (conj %1 %2) %1) numbers []))

(all-lesser-than 5 [5 7 9 3 4 1 2 8])

[3 4 1 2]

In [48]:
; A HOF 'select-if' that generalizes the 2 last functions

(defn select-if [pred elements]
    (compute-across #(if (pred %2) (conj %1 %2) %1) elements []))

#'user/select-if

In [53]:
; 'select-if' can get all the even numbers

(select-if even? [5 7 9 3 4 1 2 8])

[4 2 8]

In [52]:
; 'select-if' can get all the odd numbers

(select-if odd? [5 7 9 3 4 1 2 8])

[5 7 9 3 1]

In [54]:
; 'all-greater-than' can be implemented in terms of 'select'if'

(defn all-greater-than-v2 [threshold numbers]
    (select-if #(> % threshold) numbers))

(all-greater-than-v2 5 [5 7 9 3 4 1 2 8])

[7 9 8]

In [56]:
; 'all-lesser-than' can be implemented in terms of 'select'if'

(defn all-lesser-than-v2 [threshold numbers]
    (select-if #(< % threshold) numbers))


(all-lesser-than-v2 5 [5 7 9 3 4 1 2 8])

[3 4 1 2]

In [57]:
; In summary, 'select-if' is a custom implementation of 'filter'

(filter even? [5 7 9 3 4 1 2 8])

(4 2 8)

In [58]:
(filter odd? [5 7 9 3 4 1 2 8])

(5 7 9 3 1)

## Partial application
